<a href="https://colab.research.google.com/github/MaxGhenis/random/blob/master/warren_tax_avoidance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Warren tax avoidance

Compare the current 16% avoidance to an 8% per 1% elasticity, i.e. assume 24% avoidance for families with net worth over $1 billion who would have a 3% marginal wealth tax rate.

## Setup

TODO: Figure out how to make microdf work without installing labellines and taxcalc (optional).

In [0]:
import sys

In [2]:
!pip install matplotlib-label-lines
if 'microdf' not in sys.modules:
    !pip install git+git://github.com/maxghenis/microdf.git
if 'taxcalc' not in sys.modules:
    !pip install git+git://github.com/PSLmodels/Tax-Calculator.git

  Cloning git://github.com/maxghenis/microdf.git to /tmp/pip-req-build-oj7cc1rv
  Running command git clone -q git://github.com/maxghenis/microdf.git /tmp/pip-req-build-oj7cc1rv
  Created wheel for microdf: filename=microdf-0.1-cp36-none-any.whl size=13479 sha256=ff04e3f6b66950c3e264971fe8c72cd24b0f85d0885d7040d7d3160a1115210b
  Stored in directory: /tmp/pip-ephem-wheel-cache-948fa7nw/wheels/d0/61/d3/083d3677aa1ee6b248d229cb6c4de037f9d61b595e9f06aed0
Successfully built microdf
  Cloning git://github.com/PSLmodels/Tax-Calculator.git to /tmp/pip-req-build-3lrau328
  Running command git clone -q git://github.com/PSLmodels/Tax-Calculator.git /tmp/pip-req-build-3lrau328
  Created wheel for taxcalc: filename=taxcalc-0.0.0-cp36-none-any.whl size=60617386 sha256=5bd13348ee6f97a140a877c24dd65dd6a222237b28ec64e78d0ff02e5e8b3ea7
  Stored in directory: /tmp/pip-ephem-wheel-cache-e7h6pmin/wheels/5c/c7/c2/e2c40e13b9b3352b091aea6b0744eab7120e9094d070a7b289
Successfully built taxcalc


In [0]:
import pandas as pd
import numpy as np
import microdf as mdf

## Load data

From Saez/Zucman wealth tax calculator: https://github.com/BITSS/opa-wealthtax

In [0]:
wealth = pd.read_stata('https://github.com/BITSS/opa-wealthtax/blob/master/analysis_data/wealth.dta?raw=true')

## Descriptive analysis

Total net worth

In [5]:
total_nw = mdf.weighted_sum(wealth, 'networth', 'weight')
total_nw / 1e12

93.74638072480977

In [6]:
total_families = wealth.weight.sum()
total_families / 1e6

156.43578

In [7]:
total_ubi = (mdf.weighted_sum(wealth, 'networth_ubi', 'weight') - 
             mdf.weighted_sum(wealth, 'networth', 'weight')).sum()
total_ubi / 1e9

KeyError: ignored

### Warren wealth tax

2% above \$50 million, 3% above \$1 billion.

16% avoidance/evasion rate per Saez and Zucman, which is 2% * elasticity of 8: http://wealthtaxsimulator.org/analysis/

In [0]:
AVOID_ELASTICITY = 8

In [0]:
def tax(val, brackets, rates):
    # Args:
    #     val: Value to assess tax on, e.g. wealth or income (list or Series).
    #     brackets: Left side of each bracket (list or Series).
    #     rates: Rate corresponding to each bracket.
    df_tax = pd.DataFrame({'brackets': brackets, 'rates': rates})
    df_tax['base_tax'] = df_tax.brackets.\
        sub(df_tax.brackets.shift(fill_value=0)).\
        mul(df_tax.rates.shift(fill_value=0)).cumsum()
    rows = df_tax.brackets.searchsorted(val, side='right') - 1
    income_bracket_df = df_tax.loc[rows].reset_index(drop=True)
    return pd.Series(val).sub(income_bracket_df.brackets).\
        mul(income_bracket_df.rates).add(income_bracket_df.base_tax)

In [0]:
WARREN_RATES = [0, 0.02, 0.03]   # 0%, 2%, 3%.

WARREN_BRACKETS = [0,
                   50e6,  # First $50 million.
                   1e9]   # Over $1 billion.

In [0]:
def warren_tax(incomes):
    return income_tax(incomes, rates=WARREN_RATES, brackets=WARREN_BRACKETS)

TODO: Do this more elegantly.

In [0]:
wealth['mtr'] = np.where(wealth.networth < WARREN_BRACKETS[1], WARREN_RATES[0],
                         np.where(wealth.networth < WARREN_BRACKETS[2],
                                  WARREN_RATES[1], WARREN_RATES[2]))

### Current approach

16% constant avoidance.

In [0]:
wealth['warren_tax_base'] = ((1 - AVOID_ELASTICITY * 0.02) * 
                             np.maximum(wealth.networth, 0))

In [0]:
wealth['warren_tax'] = warren_tax(wealth.warren_tax_base)
wealth['networth_warren'] = wealth.networth - wealth.warren_tax

In [24]:
warren_revenue = (mdf.weighted_sum(wealth, 'networth_warren', 'weight') - 
                  mdf.weighted_sum(wealth, 'networth', 'weight')).sum()
warren_revenue / 1e9

-199.15986065067187

### Adjusted approach

In [0]:
wealth['warren_tax_base2'] = ((1 - AVOID_ELASTICITY * wealth.mtr) * 
                              np.maximum(wealth.networth, 0))

In [0]:
wealth['warren_tax2'] = warren_tax(wealth.warren_tax_base2)
wealth['networth_warren2'] = wealth.networth - wealth.warren_tax2

In [27]:
warren_revenue2 = (mdf.weighted_sum(wealth, 'networth_warren2', 'weight') - 
                   mdf.weighted_sum(wealth, 'networth', 'weight')).sum()
warren_revenue2 / 1e9

-189.69935268021874